### Python 基础算法练习（Hadoop + Spark + MySQL + Python）
## <center>6 >>> 面试中的算法题（C）：Python 实战</center>
### <center>算法验证：张君颖  ； 报告日期：2021.1.28</center>
  <font color=blue><center>作者邮箱：zhang.jun.ying@outlook.com</center></font>   
  
  <font color=blue><center>项目源代码、数据、自定义函数已上传GitHub：</center></font>   
    
<font color=blue><center>https://github.com/lotbear/Python-Financial-investment-strategy</center></font>

### >>> 题一：Bitmap 位图算法实现
在实际工作中，我们会用关系型数据库存（如 MySQL）储用户标签信息，但当我们用 SQL 语句检索多标签的用户统计数据时，SQL 语句就像面条一样长：   

eg: <font color=blue>Select cout(distinct Name) as 用户数 from table where Age='90后' and Occupation ='程序员' and Phone ='苹果' and ......; </font>    

为了解决这种多标签的用户检索场景，我们可以将每个标签独立成一个 Bitmap 进行位图存储。

In [1]:
class MyBitmap:
    def __init__(self,size):
        self.user_ids=[0]*(self.get_user_id_index(size-1)+1)
        self.size=size
    
    def get_bit(self,bit_index):
        if bit_index<0 or bit_index >self.size-1:
            raise Exception('超过 Bitmap 有效范围！')
        user_id_index=self.get_user_id_index(bit_index)
        return (self.user_ids[user_id_index]&(1<<bit_index))!=0
    
    def set_bit(self,bit_index):
        if bit_index<0 or bit_index >self.size-1:
            raise Exception('超过 Bitmap 有效范围！')
        user_id_index=self.get_user_id_index(bit_index)
        self.user_ids[user_id_index]|=(1<<bit_index)
        
    def get_user_id_index(self,bit_index):
        # 右移 6 位，相当于除以 64
        return bit_index>>6

In [2]:
bitMap=MyBitmap(128)
bitMap.set_bit(126)
bitMap.set_bit(16)
bitMap.set_bit(75)

print('该标签下是否存在 id = 126 的用户？',bitMap.get_bit(126))
print('该标签下是否存在 id = 18 的用户？',bitMap.get_bit(18))

该标签下是否存在 id = 126 的用户？ True
该标签下是否存在 id = 18 的用户？ False


<font color=blue> 将每个标签都以 Bitmap 存储，在进行多标签统计用户时，多个 Bitmap 进行交集 '&' or 并集 '|'位运算   
    
非运算需要再给出一个全量用户的 Bitmap，然后进行异或运算 'XOR'</font>

### >>> 题二：LRU（Least Recently Used） 算法应用
在实际工作中，为应对业务不断拓新，我们需要抽出一个用户系统，向各个业务系统提供用户的基本信息；   

如果每次检索用户信息，都是从数据库提取（如 MySQL），会影响访问性能，为此我们创建一个哈希表作为缓存，但随着用户数量激增，服务器内存消耗太快 ；  

此时需要使用 LRU 算法来管理内存，LRU 算法的基础是哈希链表，若调用户数据未在缓存中，则从数据库读取并插入哈希链表的最右侧，若查询的用户数据已在链表中，则该数据从原位置移动到链表最右侧；   

当哈希链表的总长度达到预定的阈值，新进一个用户缓存数据，就需要从链表的最左端剔除一个用户数据（很久没有调用过的用户数据）。   

虽然 Python 中的 collections.OrderedDict 已经对哈希链表做了很好的实现，我们还是自己再用代码简单实现一下：

In [3]:
class LRUCache:
    def __init__(self, limit):
        self.limit = limit
        self.hash = {}
        self.head = None
        self.end = None

    def get(self, key):
        node = self.hash.get(key)
        if node is None:
            return None
        self.refresh_node(node)
        return node.value

    def put(self, key, value):
        node = self.hash.get(key)
        if node is None:
            # 如果key不存在，插入key-value
            if len(self.hash) >= self.limit:
                old_key = self.remove_node(self.head)
                self.hash.pop(old_key)
            node = Node(key, value)
            self.add_node(node)
            self.hash[key] = node
        else:
            # 如果key存在，刷新key-value
            node.value = value
            self.refresh_node(node)

    def remove(self, key):
        node = self.hash.get(key)
        if node is None:
            return
        self.remove_node(node)
        self.hash.remove(key)

    def refresh_node(self, node):
        # 如果访问的是尾节点，无需移动节点
        if node == self.end:
            return
        # 移除节点
        self.remove_node(node)
        # 重新插入节点
        self.add_node(node)

    def remove_node(self, node):
        if node == self.head and node == self.end:
            # 移除唯一的节点
            self.head = None
            self.end = None
        elif node == self.end:
            # 移除节点
            self.end = self.end.pre
            self.end.next = None
        elif node == self.head:
            # 移除头节点
            self.head = self.head.next
            self.head.pre = None
        else:
            # 移除中间节点
            node.pre.next = node.pre
            node.next.pre = node.pre
        return node.key

    def add_node(self, node):
        if self.end is not None:
            self.end.next = node
            node.pre = self.end
            node.next = None
        self.end = node
        if self.head is None:
            self.head = node


class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.pre = None
        self.next = None

In [4]:
lruCache=LRUCache(5)
lruCache.put('001','用户 1 信息')
lruCache.put('002','用户 2 信息')
lruCache.put('003','用户 3 信息')
lruCache.put('004','用户 4 信息')
lruCache.put('005','用户 5 信息')
print(lruCache.get('003'))

lruCache.put('004','用户 4 信息更新')
lruCache.put('005','用户 6 信息')
print(lruCache.get('001'))
print(lruCache.get('006'))

用户 3 信息
用户 1 信息
None


<font color=blue>对于用户系统的需求，可以用缓存数据库 Redis 实现，Redis 底层也实现了类似 LRU 的回收算法。

### >>> 题三：随机红包算法
1.所有人抢到的金额之和要等于红包金额   

2.保证红包拆分的金额尽可能均匀分布，不能出现严重的两极分化现象  

思路：把每次红包随机金额的上限定为剩余人均金额的 2 倍。

In [5]:
import random
# random.uniform 可随机生成浮点数，但精度无法控制，可能导致红包总金额出现误差
# 我们选择 ranint 作为随机函数，为了最终生成带两位小数的红包金额
# 我们先将 total_amount*100 ，输出时再 my_amount/100

def divide_red_package(total_amount, total_people_num):
    amount_list = []
    rest_amount = total_amount*100
    rest_people_num = total_people_num
    for i in range(0, total_people_num-1):
        # 随机范围：[1，剩余人均金额的两倍)，左闭右开
        amount = random.randint(1,int(rest_amount/rest_people_num*2)-1)
        rest_amount -= amount
        rest_people_num -= 1
        amount_list.append(amount)
    amount_list.append(rest_amount)
    return amount_list

In [6]:
my_amount_list = divide_red_package(99, 10)
print('总红包金额: %.2f' % (sum(my_amount_list)/100),'元')
print('='*50)
for my_amount in my_amount_list:
    print('抢到金额: %.2f' % (my_amount/100),'元')

总红包金额: 99.00 元
抢到金额: 14.07 元
抢到金额: 1.36 元
抢到金额: 10.71 元
抢到金额: 9.59 元
抢到金额: 7.99 元
抢到金额: 16.08 元
抢到金额: 14.92 元
抢到金额: 12.40 元
抢到金额: 3.43 元
抢到金额: 8.45 元


番外：对多个list的对应元素求和

In [7]:
def list_add(a,b):
    c = []
    for i in range(len(a)):
        c.append(a[i]+b[i])
    return c
 
if __name__ == '__main__':
    a = [1,2,3]
    b = [2,3,4]
    c = [3,4,5]
    print(list_add(list_add(a,b),c))

[6, 9, 12]


In [8]:
import numpy as np
a = np.array([1,2,3])
b = np.array([2,3,4])
c = np.array([3,4,5])
print(a+b+c)

[ 6  9 12]


In [9]:
import numpy as np
a = [1,2,3]
b = [2,3,4]
c = [3,4,5]
print(np.sum([a,b,c], axis = 0))

[ 6  9 12]
